In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

In [6]:
df = pd.read_csv('cleaned_data.csv')

# 获取发射点列表
launch_sites = df['LaunchSite'].unique()

# 创建Dash应用
app = dash.Dash(__name__)

# 设计仪表板布局
app.layout = html.Div([
    html.H1("Rocket Launch Dashboard"),
    
    # 下拉菜单
    dcc.Dropdown(
        id='site-dropdown',
        options=[{'label': site, 'value': site} for site in launch_sites],
        value=launch_sites[0],
        multi=False,
        style={'width': '50%'}
    ),
    
    # 成功率文本显示
    html.Div(id='success-rate-output'),

    # 饼图
    dcc.Graph(id='success-pie-chart')
])

# 回调函数，根据下拉菜单选择的发射地点更新成功率文本和饼图
@app.callback(
    [Output('success-rate-output', 'children'),
     Output('success-pie-chart', 'figure')],
    [Input('site-dropdown', 'value')]
)
def update_success_rate(selected_site):
    # 根据选择的发射地点过滤数据
    filtered_data = df[df['LaunchSite'] == selected_site]

    # 计算成功率
    success_rate = filtered_data['Outcome'].mean() * 100

    # 成功率文本显示
    success_rate_text = f"Success Rate at {selected_site}: {success_rate:.2f}%"

    # 创建饼图
    fig = px.pie(names=['Success', 'Failure'], values=[success_rate, 100 - success_rate], title=success_rate_text)

    return success_rate_text, fig

# 运行Dash应用
if __name__ == '__main__':
    app.run_server(debug=True)

In [11]:
min_payload = int(df['PayloadMass'].min())
max_payload = int(df['PayloadMass'].max())

# 创建Dash应用
app = dash.Dash(__name__)

# 设计仪表板布局
app.layout = html.Div([
    html.H1("Payload Analysis Dashboard"),
    
    # Payload范围选择器
    dcc.RangeSlider(
        id='payload-range-slider',
        min=min_payload,
        max=max_payload,
        marks={i: str(i) for i in range(min_payload, max_payload+1, 50)},
        step=10,
        value=[df['PayloadMass'].min(), df['PayloadMass'].max()]
    ),
    
    # Payload负载与着陆结果的散点图
    dcc.Graph(id='payload-landing-scatter'),
    
    # Payload负载与所有发射地点的散点图
    dcc.Graph(id='payload-site-scatter')
])

# 回调函数，根据Payload范围更新散点图
@app.callback(
    [Output('payload-landing-scatter', 'figure'),
     Output('payload-site-scatter', 'figure')],
    [Input('payload-range-slider', 'value')]
)
def update_scatter_plots(payload_range):
    # 根据Payload范围过滤数据
    filtered_data = df[(df['PayloadMass'] >= payload_range[0]) & (df['PayloadMass'] <= payload_range[1])]

    # Payload负载与着陆结果的散点图
    scatter_fig1 = px.scatter(
        filtered_data,
        x='PayloadMass',
        y='Outcome',
        color='Outcome',
        labels={'Payload Mass': 'Payload Mass', 'Outcome': 'Landing Outcome'},
        title='Payload Mass vs. Landing Outcome'
    )

    # Payload负载与所有发射地点的散点图
    scatter_fig2 = px.scatter(
        filtered_data,
        x='PayloadMass',
        y='LaunchSite',
        color='Outcome',
        labels={'PayloadMass': 'PayloadMass', 'LaunchSite': 'LaunchSite'},
        title='Payload Mass vs. Launch Site'
    )

    return scatter_fig1, scatter_fig2

# 运行Dash应用
if __name__ == '__main__':
    app.run_server(debug=True)